In [7]:
%load_ext autoreload
%autoreload 2

import os
os.chdir("/nfs/turbo/coe-chaijy/sstorks/simulation_informed_pcr4nlu/TRAVEl")
from travel import init_travel
init_travel()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load preds

In [8]:
import json

# RESULTS_PATH = "/home/sstorks/coe-chaijy/sstorks/simulation_informed_pcr4nlu/TRAVEl/saved_results_222/vqa_mistake_detection/ego4d/SuccessVQA_ego4d_debug250_llava-1.5-7b-hf_20240701113527/preds_heuristic_val.json"
RESULTS_PATH = "/home/sstorks/coe-chaijy/sstorks/simulation_informed_pcr4nlu/TRAVEl/saved_results_222/vqa_mistake_detection/ego4d/VQG2VQA_ego4d_debug250_llava-1.5-7b-hf_20240701115231/preds_heuristic_val.json"
preds = json.load(open(RESULTS_PATH, "r"))

# Visualize some outputs

In [10]:
import random

output_indices_to_visualize = random.sample(list(range(len(preds))), 20)

print("Indices to visualize:", output_indices_to_visualize)

Indices to visualize: [36, 488, 251, 342, 263, 104, 187, 358, 127, 261, 172, 497, 224, 226, 100, 460, 118, 461, 438, 32]


In [ ]:
import json
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
from PIL import Image
from pprint import pprint

from travel.data.ego4d import Ego4DMistakeDetectionDataset
from travel.data.mistake_detection import MistakeDetectionExample, get_cutoff_time_by_proportion
from travel.model.vqa import VQAResponse
from travel.model.mistake_detection import aggregate_mistake_probs_over_frames
from travel.model.mistake_detection import DETECTION_FRAMES_PROPORTION

show_frames = True

# Gather up example dirs by example ID
dataset_cache_dir = Ego4DMistakeDetectionDataset.get_cache_dir("val",
                                                         mismatch_augmentation=True,
                                                         multi_frame=True,
                                                         debug_n_examples_per_class=250)
dataset_example_dirs = json.load(open(os.path.join(dataset_cache_dir, "dataset.json"), "r"))["example_dirs"]
dataset_example_dirs = {"/".join(d.split("/")[-3:]): d for d in dataset_example_dirs}

keys = list(preds.keys())
for output_idx in output_indices_to_visualize:
    print(f"Displaying output index {output_idx}\n")
    pred = preds[keys[output_idx]]

    procedure_description = pred['example']['procedure_description']
    mistake = pred["example"]["mistake"]
        
    example_id = pred['example']['example_id']
    example = Ego4DMistakeDetectionDataset.load_example_from_file(dataset_example_dirs[example_id])
    example.cutoff_to_last_frames(DETECTION_FRAMES_PROPORTION)
    pprint(example)
    assert len(example.frames) == len(pred['vqa'])

    # Extract the vqa list - just take the questions and answers for last frame
    # Create a figure with a subplot for each entry
    num_entries = len(pred['vqa'])
    num_questions = len(pred['vqa'][0])
    if show_frames:
        fig, axs = plt.subplots(num_entries, num_questions, figsize=(10 * num_questions, 5 * num_entries))
    print(f"{len(pred['vqa'])} frames")
    for frame_idx in range(len(pred['vqa'])):
        vqa_list = pred['vqa'][frame_idx]

        for i, entry in enumerate(vqa_list):
            if show_frames:
                ax = axs[frame_idx, i] if num_questions > 1 else axs[frame_idx]
                img = Image.open(entry["frame"]) if "frame" in entry and entry["frame"] != "" else example.frames[frame_idx]
                ax.imshow(img)
                ax.axis('off')

            question = entry['question']
            prompt = entry['prompt'].split("USER: <image>")[1].split("ASSISTANT:")[0].strip()
            predicted_answer = entry['predicted_answer']
            expected_answer = entry['expected_answer']

            title = f"Procedure: {procedure_description} ({'mistake' if mistake else 'success'}: {pred['example']['mistake_type']})\n"
            title += f"Q: {question} ({round(example.frame_times[frame_idx], 1)} sec.)\n"
            title += f"Transformed Q: {prompt}\n"
            if i < len(pred['mistake_detection']['0.0']['mistake_probs'][frame_idx]):
                mistake_prob = round(pred['mistake_detection']['0.0']['mistake_probs'][frame_idx][i], 3)
            else:
                mistake_prob = 1.0
            title += f"Predicted Answer: {VQAResponse(predicted_answer).name if mistake_prob != 1.0 else VQAResponse(1 - expected_answer).name} ({mistake_prob if mistake_prob == 1.0 or predicted_answer != expected_answer else 1 - mistake_prob}), Expected Answer: {VQAResponse(expected_answer).name}"

            if show_frames:
                ax.set_title(title)

    if show_frames:
        plt.tight_layout()
        plt.show()

    # Print out probabilities
    for k in ["mistake_probs", "nli_relevance_probs", "nli_mistake_probs"]:
        print(k + ":")
        pprint(pred['mistake_detection']["0.0"][k])
        
    frame_times = pred['example']['frame_times']
    cutoff_time = get_cutoff_time_by_proportion(frame_times, DETECTION_FRAMES_PROPORTION)
    frame_times = [t for t in frame_times if t >= cutoff_time]

    print("\nMISTAKE CONFIDENCE AGGREGATION:")
    mistake_prob = aggregate_mistake_probs_over_frames(pred['mistake_detection']["0.0"]['mistake_probs'], frame_times, verbose=False)
    print(mistake_prob)


In [40]:
hose_count = 0
all_count = 0
for pred in preds.values():
    if "hose" in pred['example']['procedure_description']:
        hose_count += 1
    all_count += 1
        
print(f"{hose_count}/{all_count} examples are about hoses")

4/500 examples are about hoses
